In [134]:
import numpy as np
DTYPE = np.float64
epsilon = 1e-4

import matplotlib.pyplot as plt

import pandas as pd

from tqdm import tqdm

import os

In [135]:
def floryHuggins_2components(phi:DTYPE, chi:DTYPE):
    f = phi*np.log(phi) + (1-phi)*np.log(1-phi) + chi*phi*(1-phi)
    return f

In [147]:
class metropolis:
    def __init__(self, beta, phi_global, chi):
        self.beta = beta
        self.phi_global = phi_global
        self.chi = chi
        
    def metropolisStep(self, current_phi_11, current_eta_1):
        current_phi_12 = (self.phi_global - current_eta_1*current_phi_11)/(1-current_eta_1)
        
        proposal_phi_11 = round(np.random.uniform(0+epsilon, 1-epsilon), 4)
        proposal_eta_1 = round(np.random.uniform(0+epsilon, 1-epsilon), 4)

        proposal_phi_12 = (self.phi_global - proposal_eta_1*proposal_phi_11)/(1-proposal_eta_1)
        
        if proposal_phi_12 <= 1 and proposal_phi_12 >= 0:
            f_1_current = floryHuggins_2components(current_phi_11, self.chi) 
            f_2_current = floryHuggins_2components(current_phi_12, self.chi) 

            f_current = current_eta_1*f_1_current  + (1-current_eta_1)*f_2_current


            f_1_proposal = floryHuggins_2components(proposal_phi_11, self.chi)
            f_2_proposal = floryHuggins_2components(proposal_phi_12, self.chi)
            
            f_proposal = proposal_phi_11*f_1_proposal + proposal_phi_12*f_2_proposal

            delta_f = f_proposal - f_current

            if delta_f < 0 or np.random.uniform(0, 1) < np.exp(-self.beta*delta_f):
                return [proposal_phi_11, proposal_eta_1]
            else:
                return [current_phi_11, current_eta_1]
        else:
            return [current_phi_11, current_eta_1]
        
    def simulate(self, start_phi_11, start_eta_1, nSteps):
        walk = []
        walk.append([start_phi_11, start_eta_1])
            
        for i in range(nSteps):
            [new_phi_11, new_eta_1] = self.metropolisStep(start_phi_11, start_eta_1)
            walk.append([new_phi_11, new_eta_1])

            start_phi_11, start_eta_1 = new_phi_11, new_eta_1
            # print([new_phi_11, new_eta_1])
        
        return walk

In [148]:
m = metropolis(1, 0.223, 2.778)

In [149]:
walk = m.simulate(0.1, 0.1, 100000)

In [142]:
walk

[[0.1, 0.1],
 [0.645, 0.2473],
 [0.645, 0.2473],
 [0.645, 0.2473],
 [0.645, 0.2473],
 [0.2054, 0.095],
 [0.2054, 0.095],
 [0.3632, 0.2534],
 [0.3632, 0.2534],
 [0.3632, 0.2534],
 [0.3632, 0.2534],
 [0.0508, 0.0903],
 [0.0162, 0.6735],
 [0.0162, 0.6735],
 [0.2555, 0.3066],
 [0.2555, 0.3066],
 [0.2555, 0.3066],
 [0.8482, 0.1437],
 [0.6815, 0.3245],
 [0.5102, 0.3582],
 [0.1428, 0.8367],
 [0.4974, 0.2135],
 [0.1769, 0.0109],
 [0.1769, 0.0109],
 [0.1769, 0.0109],
 [0.1769, 0.0109],
 [0.806, 0.2385],
 [0.806, 0.2385],
 [0.1846, 0.8784],
 [0.3135, 0.0778],
 [0.6186, 0.1309],
 [0.0074, 0.7002],
 [0.8118, 0.2488],
 [0.7137, 0.06],
 [0.7137, 0.06],
 [0.7137, 0.06],
 [0.1394, 0.2163],
 [0.1394, 0.2163],
 [0.7569, 0.0281],
 [0.3115, 0.7081],
 [0.0314, 0.0775],
 [0.2233, 0.7568],
 [0.0378, 0.6235],
 [0.1467, 0.2915],
 [0.1467, 0.2915],
 [0.1467, 0.2915],
 [0.1467, 0.2915],
 [0.1467, 0.2915],
 [0.1467, 0.2915],
 [0.1467, 0.2915],
 [0.1467, 0.2915],
 [0.1467, 0.2915],
 [0.1152, 0.8034],
 [0.0592, 0.7

In [153]:
fig, ax = plt.subplots(figsize = (6, 6))

for idx, _ in tqdm(enumerate(walk[::10])):
    ax.scatter(_[0], _[1], color = "k", s = 5, alpha = 0.1)

# ax.xaxis.set_inverted(True)
ax.yaxis.set_inverted(True)

ax.set_xlabel(r"$\phi_{11}$")
ax.set_ylabel(r"$\eta_1$")
title = r"$\phi_1^{\text{global}}$ = " + f"{0.223:.3f}" + "\n" + r"$\chi = $" + f"{2.778:.3f}"
fig.suptitle(title)

fig.tight_layout()

output_filepath = f"data_mcmc/chi-{2.778:.3f}"
output_filename = f"phi_g-{0.223:.3f}.png"

if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

file = os.path.join(output_filepath, output_filename)     

plt.savefig(file, dpi = 400)
plt.close()

print(f"Saved @ {file}")


10001it [00:48, 206.44it/s]


Saved @ data_mcmc/chi-2.778/phi_g-0.223.png
